<p align="center">
  <img src="../logo_ufjf.png" width="200"><br>
  <b>UNIVERSIDADE FEDERAL DE JUIZ DE FORA</b><br>
  Programa de Pós-Graduação em Matemática<br>
  Análise Numérica I<br><br>
  Luís Karlos Mendes de Oliveira<br><br>
  <b>Atividade 02:</b> Métodos Numéricos<br><br>
  <p align="left">Prof.: Sandro Rodrigues Mazorche</p>
  <p align="right">29 de setembro de 2025</p>
</p>


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import sympy as sp
import pandas as pd
import mylib as m
import os
import ipynbname

diretorio_tabela = f"tables/{ipynbname.name()}"
diretorio_imagem = f"img/{ipynbname.name()}"
os.makedirs(diretorio_tabela, exist_ok=True)
os.makedirs(diretorio_imagem, exist_ok=True)


## **Questão 2**
$\begin{cases}
  x'(t) = rx(t)(1-x(t)) - \dfrac{x^2(t)}{1 + x^2(t)} \\
  x(0) = 0.1 \\
  t \in [0, 10]
\end{cases}$


In [ ]:
f = lambda t, x: r*x*(1 - x) - (x**2)/(1 + x**2)
r = 6
condicao_inicial = [0, .1]

def x_exato(t):
    T = max(t) if isinstance(t, (np.ndarray, list)) else t
    dt = 0.001
    t_rk, x_rk = m.metodo_rk6(f, condicao_inicial, T, dt)

    return np.interp(t, t_rk, x_rk)

T = 10
h = [.5, .25, .125]
tempos_desejados = [1.5, 3, 6, 9]


### Método de Euler Explícito

In [ ]:
resultados_mee = {}
for dt in h:
  print(f'Resolvendo o PVI com h = {dt} ...')
  t_euler, y_euler = m.metodo_euler_explicito(f, condicao_inicial, T, dt)

  dados = pd.DataFrame({
    't': t_euler,
    'x(t)': y_euler,
  })
  resultados_mee[f'h={dt}'] = dados

fig, ax = plt.subplots(figsize=(12, 7))
for nome, df in resultados_mee.items():
    ax.plot(df['t'], df['x(t)'], '.-', label=f'{nome}')
ax.set_xlabel('t', fontsize=14)
ax.set_ylabel('x(t)', fontsize=14)
ax.legend(fontsize=14, loc='lower right')
ax.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.autoscale(enable=True, axis='x', tight=True)
nome_arquivo = f"{diretorio_imagem}/euler_r_{r}.pdf"
plt.savefig(nome_arquivo)
print(f"✅ Gráfico salvo com sucesso no arquivo: {nome_arquivo}")
plt.show()


### Método Runge-Kutta

In [ ]:
resultados_mrk = {}
metodos = {'RK2': m.metodo_rk2, 'RK4': m.metodo_rk4}
for nome_metodo, metodo in metodos.items():
    for dt in h:
        t_rk, x_rk = metodo(f, condicao_inicial, T, dt)
        dados = pd.DataFrame({
            't': t_rk,
            'x(t)': x_rk,
        })
        resultados_mrk[f'{nome_metodo} h={dt}'] = dados

for nome_metodo in metodos.keys():
    fig, ax = plt.subplots(figsize=(12, 7))
    for k, df in resultados_mrk.items():
        if k.startswith(nome_metodo):
            ax.plot(df['t'], df['x(t)'], '.-', label=f'{k}')
    ax.set_xlabel('t', fontsize=14)
    ax.set_ylabel('x(t)', fontsize=14)
    ax.legend(fontsize=14, loc='lower right')
    ax.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.autoscale(enable=True, axis='x', tight=True)
    nome_arquivo = f"{diretorio_imagem}/rk_{nome_metodo.lower()}_r_{r}.pdf"
    plt.savefig(nome_arquivo)
    print(f"✅ Gráfico {nome_metodo} salvo com sucesso no arquivo: {nome_arquivo}")
    plt.show()


### Runge-Kutta adaptativo

In [ ]:
resultados_mrk_adapt = {}
metodos_adapt = {'RKF45': 'RKF45','DOPRI54': 'DOPRI54'}

for nome_metodo, tipo in metodos_adapt.items():
    t_rk, x_rk, n, i = m.metodo_rk(f, condicao_inicial, T, 1, 1e-6, tipo)
    y_exato = x_exato(t_rk)
    dados = pd.DataFrame({
        't': t_rk,
        'x(t)': x_rk,
    })
    resultados_mrk_adapt[nome_metodo] = dados
    print(f'✅ {nome_metodo}: Número de pontos: {n}, Número de iterações: {i}')


cores = ['tab:blue', 'tab:orange']
fig, axs = plt.subplots(1, 2, figsize=(18, 7))
for i, (nome, df) in enumerate(resultados_mrk_adapt.items()):
    ax = axs[i]
    ax.plot(df['t'], df['x(t)'], '.-', color=cores[i], label=nome)
    ax.set_xlabel('t', fontsize=14)
    ax.set_ylabel('x(t)', fontsize=14)
    ax.legend(fontsize=14, loc='lower right')
    ax.grid(True, which='both', linestyle='--', linewidth=0.5)
    ax.autoscale(enable=True, axis='x', tight=True)
nome_arquivo = f"{diretorio_imagem}/rk_adaptativos_r_{r}.pdf"
plt.tight_layout()
plt.savefig(nome_arquivo)
print(f"✅ Gráfico adaptativos salvo com sucesso no arquivo: {nome_arquivo}")
plt.show()


In [ ]:
# Calcular x(t) nos tempos desejados usando RKF45 e DOPRI54
resultados_mrk_adapt = {}
metodos_adapt = {'RKF45': 'RKF45', 'DOPRI54': 'DOPRI54'}

for nome_metodo, tipo in metodos_adapt.items():
    x_adapt = []
    for t_val in tempos_desejados:
        t_rk, x_rk, n, i = m.metodo_rk(f, condicao_inicial, t_val, 1, 1e-6, tipo)
        x_adapt.append(x_rk[-1])
    resultados_mrk_adapt[nome_metodo] = pd.DataFrame({'t': tempos_desejados, 'x(t)': x_adapt})
    print(f'✅ {nome_metodo}: x(t) nos tempos desejados:', x_adapt)


In [ ]:
for dt in h:
    linhas = []
    for t_val in tempos_desejados:
        linha = {'t': t_val}
        # Euler
        nome_mee = f'h={dt}'
        if nome_mee in resultados_mee:
            df_mee = resultados_mee[nome_mee]
            val_mee = df_mee.loc[df_mee['t'].round(5) == round(t_val,5), 'x(t)']
            linha['Euler'] = val_mee.values[0] if not val_mee.empty else None
        else:
            linha['Euler'] = None
        # RK2
        nome_rk2 = f'RK2 h={dt}'
        if nome_rk2 in resultados_mrk:
            df_rk2 = resultados_mrk[nome_rk2]
            val_rk2 = df_rk2.loc[df_rk2['t'].round(5) == round(t_val,5), 'x(t)']
            linha['RK2'] = val_rk2.values[0] if not val_rk2.empty else None
        else:
            linha['RK2'] = None
        # RK4
        nome_rk4 = f'RK4 h={dt}'
        if nome_rk4 in resultados_mrk:
            df_rk4 = resultados_mrk[nome_rk4]
            val_rk4 = df_rk4.loc[df_rk4['t'].round(5) == round(t_val,5), 'x(t)']
            linha['RK4'] = val_rk4.values[0] if not val_rk4.empty else None
        else:
            linha['RK4'] = None
        # RKF45 adaptativo
        if 'RKF45' in resultados_mrk_adapt:
            df_rkf45 = resultados_mrk_adapt['RKF45']
            val_rkf45 = df_rkf45.loc[df_rkf45['t'].round(5) == round(t_val,5), 'x(t)']
            linha['RKF45'] = val_rkf45.values[0] if not val_rkf45.empty else None
        else:
            linha['RKF45'] = None
        # DOPRI54 adaptativo
        if 'DOPRI54' in resultados_mrk_adapt:
            df_dopri = resultados_mrk_adapt['DOPRI54']
            val_dopri = df_dopri.loc[df_dopri['t'].round(5) == round(t_val,5), 'x(t)']
            linha['DOPRI54'] = val_dopri.values[0] if not val_dopri.empty else None
        else:
            linha['DOPRI54'] = None
        linhas.append(linha)
    tabela = pd.DataFrame(linhas)
    pd.set_option('display.float_format', lambda x: f'{x:.9f}')
    print(f'\nTabela comparativa para h = {dt}:')
    display(tabela)
    # Exporta para LaTeX
    tabela_latex = tabela.fillna('-')
    nome_arquivo = f"{diretorio_tabela}/tabela_comparativa_h_{dt}_r_{r}.tex"
    codigo_latex = tabela_latex.to_latex(
        index=False,
        escape=True,
        float_format='%.9f'.__mod__,
        caption=f"Comparação dos métodos para h = {dt} e r = {r} nos tempos desejados.",
        label=f"tab:comparativa_h_{dt}_r_{r}",
        column_format='ccccccc',
        na_rep='-'
    )
    codigo_latex = (
        "\\begin{table}[H]\n"
        "\\centering\n"
        + codigo_latex.replace("\\begin{table}", "").replace("\\end{table}", "")
        + "\\end{table}"
    )
    with open(nome_arquivo, 'w') as file:
        file.write(codigo_latex)
    print(f"✅ Tabela LaTeX salva em: {nome_arquivo}")
